In [ ]:
cd /Users/oleg.vlasovetc/Public/GGLasso

In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import itertools
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn.covariance import empirical_covariance
from sklearn import set_config
set_config(print_changed_only=False)

from gglasso.solver.single_admm_solver import ADMM_SGL
from gglasso.solver.single_admm_solver import block_SGL
from gglasso.helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from gglasso.helper.model_selection import single_grid_search
from experiments.benchmarks.benchmarks import *

from regain.covariance import GraphicalLasso as rg_GL

In [ ]:
p = 100
N = 200
Omega_0 = np.eye(p)
lambda1 = 0.01
# tol_list = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
tol_list = [1e-4, 1e-5, 1e-6]
enet_list = [1e-4, 1e-5, 1e-6]

Sigma, Theta = group_power_network(p, K=5, M=2)  # Theta is true precision matrix

S, samples = sample_covariance_matrix(Sigma, N)

S = S[0, :, :]
Theta = Theta[0, :, :]  # true precision matrix
X = samples[0,:,:].T

# Model solution Z

In [ ]:
Z, Z_time = model_solution(model="sklearn", X=X, lambda1=lambda1, max_iter=50000, tol=1e-7, enet=0.1)

In [ ]:
print(Z_time)

# Solution Z_i

In [ ]:
models = models_to_dict(models=["sklearn", "regain"], lambda1=lambda1, tol_list=tol_list, rtol_list=tol_list, enet_list=enet_list, 
                        max_iter=50000)

In [ ]:
sk_time, sk_accuracy, Z_sk = sklearn_time_benchmark(models, X=X, Z=Z, n_iter=5)

In [ ]:
admm_time, admm_accuracy, Z_admm = admm_time_benchmark(S=S, Omega_0=Omega_0, Z=Z, lambda1=lambda1, 
                                               method_list=["single", "block"],
                                               stop_list=['boyd', 'kkt'], 
                                               tol_list=tol_list, 
                                               rtol_list=tol_list,
                                               n_iter=5)

# Data formating

In [ ]:
sparsity_sk = list(map(gini, Z_sk.values()))
sparsity_admm =  list(map(gini, Z_admm.values()))

In [ ]:
df_sk = pd.DataFrame(data={'name': sk_time.keys(), 'time': sk_time.values(), "accuracy": sk_accuracy.values(),
                          "sparsity": sparsity_sk})

df_admm = pd.DataFrame(data={'name': admm_time.keys(), 'time': admm_time.values(), "accuracy": admm_accuracy.values(),
                            "sparsity": sparsity_admm})

df = pd.concat([df_sk, df_admm])
df

In [ ]:
df['split'] = df['name'].str.split('_')
df[["method", "tol_str", "tol", "rtol_str", "rtol"]] = pd.DataFrame(df['split'].tolist(), index=df['split'].index)
df = df.drop(["name", 'split', "tol_str", "rtol_str"], axis=1)
df

Here in sklearn, "rtol" corresponds to "enet_tol", see [sklearn.covariance.graphical_lasso](https://scikit-learn.org/stable/modules/generated/sklearn.covariance.graphical_lasso.html)

In [ ]:
fig = px.line(df, x="time", y="accuracy", text = "tol", color= "method", 
              log_y = True, 
              labels={
                     "time": "Time, s",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
fig = px.line(df, x="tol", y="accuracy", color= "method", 
              log_y = True,
              labels={
                     "tol": "Tolerance rate",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()

# Sparsity benchmark

In [ ]:
fig = go.Figure(data=go.Heatmap(
        z=df["sparsity"],
        x=df["method"],
        y=df["tol"],
        colorscale='Viridis'))

fig.update_layout(
    title='Sparsity benchmarks',
    xaxis_nticks=36)

fig.show()